In [4]:
import tensorflow as tf
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import cv2
import tools
import math
import h5py
import pathlib
import tools.datasets.tfrecord_helper as tfrh
import progressbar

In [7]:
__dataset_location = r"E:\MasterDaten\Datasets\FHAD"

In [18]:
skeleton_location = os.path.join(__dataset_location, "Hand_pose_annotation_v1")
skeleton_files = [file for file in pathlib.Path(skeleton_location).rglob("skeleton.txt")]

train_subjects = ["Subject_1", "Subject_3", "Subject_4"]
validate_subjects = ["Subject_2"]

train_skel_files = []
val_skel_files = []
test_skel_files = []
for skeleton_file in skeleton_files:
    relative_path = pathlib.Path(os.path.relpath(skeleton_file, skeleton_location))
    subject = relative_path.parts[0]
    if subject in train_subjects:
        train_skel_files.append(skeleton_file)
    elif subject in validate_subjects:
        val_skel_files.append(skeleton_file)
    else:
        test_skel_files.append(skeleton_file)

all_skel_files = np.array([train_skel_files, val_skel_files, test_skel_files])
print(all_skel_files)

[list([WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/1/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/2/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/3/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/charge_cell_phone/4/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/clean_glasses/1/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/clean_glasses/2/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/clean_glasses/3/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/clean_glasses/4/skeleton.txt'), WindowsPath('E:/MasterDaten/Datasets/FHAD/Hand_pose_annotation_v1/Subject_1/close_juice_bottle/1/skeleton.txt'), 

In [23]:
subsets = ["train", "validation", "test"]
max_chunk_size = 5000
img_basepath = os.path.join(__dataset_location, "Video_files")
tfrecord_basepath = os.path.join(__dataset_location, "tfrecord_data")
progbar = None

for idx, subset in enumerate(subsets):
    sample_idx = 0
    chunk_idx = 0
    
    if progbar: del progbar
    progbar = progressbar.ProgressBar(widgets=[progressbar.Bar(), progressbar.Percentage(), progressbar.ETA()])
    progbar.start(max_value=len(all_skel_files[idx]))
    
    for file_idx, file in enumerate(all_skel_files[idx]):
        
        skels_world_coords = np.loadtxt(file)
        for line in skels_world_coords:
            frame_index = int(line[0])
            skeleton = line[1:]
            
            img_filename = "depth_{:04d}.png".format(frame_index)
            img_path = os.path.join(img_basepath, img_filename)
            
            with tf.io.gfile.GFile(img_path, 'rb') as fid:
                img = fid.read()

            if sample_idx % max_chunk_size == 0:
                # write data to chunk file
                filename = "{}_{}.tfrecord".format(subset, chunk_idx)
                record_writer = tf.io.TFRecordWriter(os.path.join(tfrecord_basepath, filename))
                chunk_idx += 1

            tfrecord_str = tfrh.make_standard_pose_record(sample_idx, img, 640, 480, skeleton)
            record_writer.write(tfrecord_str)
            sample_idx += 1
            
        progbar.update(file_idx)
    


KeyboardInterrupt: 